In [1]:
training_list = []
testing_list = []
unsupervised_list = []

with open("resources/train-pos.txt",encoding='utf-8') as fid:
    for line in fid:
        training_list.append(line)
        
with open("resources/train-neg.txt",encoding='utf-8') as fid:
    for line in fid:
        training_list.append(line)
        
with open("resources/test-pos.txt",encoding='utf-8') as fid:
    for line in fid:
        testing_list.append(line)

with open("resources/test-neg.txt",encoding='utf-8') as fid:
    for line in fid:
        testing_list.append(line) 
        
with open("resources/train-unsup.txt",encoding='utf-8') as fid:
    for line in fid:
        unsupervised_list.append(line) 

In [2]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []
count = 0

for document in training_list + testing_list + unsupervised_list:
    sentences.append(document.split())
    
print('Word2Vec generation started ...')
model = gensim.models.Word2Vec(sentences, min_count=5, window=10, size=100, workers=7)
print('Saving the model')
model.save('resources/imdb.w2v')
model = gensim.models.Word2Vec.load('resources/imdb.w2v')
model.wv.save_word2vec_format('resources/imdb_readable.w2v')

C:\Users\Yash\Anaconda3\lib\site-packages\gensim-1.0.1-py3.6-win-amd64.egg\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-04-03 09:29:56,935 : INFO : collecting all words and their counts
2017-04-03 09:29:56,937 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Word2Vec generation started ...


2017-04-03 09:29:57,716 : INFO : PROGRESS: at sentence #10000, processed 2390666 words, keeping 49770 word types
2017-04-03 09:29:58,422 : INFO : PROGRESS: at sentence #20000, processed 4731654 words, keeping 67469 word types
2017-04-03 09:29:59,170 : INFO : PROGRESS: at sentence #30000, processed 7088415 words, keeping 80632 word types
2017-04-03 09:29:59,976 : INFO : PROGRESS: at sentence #40000, processed 9381192 words, keeping 91668 word types
2017-04-03 09:30:00,783 : INFO : PROGRESS: at sentence #50000, processed 11708957 words, keeping 100673 word types
2017-04-03 09:30:01,505 : INFO : PROGRESS: at sentence #60000, processed 14109451 words, keeping 109920 word types
2017-04-03 09:30:02,288 : INFO : PROGRESS: at sentence #70000, processed 16471157 words, keeping 118004 word types
2017-04-03 09:30:03,313 : INFO : PROGRESS: at sentence #80000, processed 18840695 words, keeping 125686 word types
2017-04-03 09:30:04,084 : INFO : PROGRESS: at sentence #90000, processed 21221705 words,

Saving the model


2017-04-03 09:33:52,198 : INFO : saved C:/Users/Yash/Documents/Project_4/imdb1.w2v
2017-04-03 09:33:52,202 : INFO : loading Word2Vec object from C:/Users/Yash/Documents/Project_4/imdb1.w2v
2017-04-03 09:33:54,656 : INFO : loading wv recursively from C:/Users/Yash/Documents/Project_4/imdb1.w2v.wv.* with mmap=None
2017-04-03 09:33:54,658 : INFO : setting ignored attribute syn0norm to None
2017-04-03 09:33:54,661 : INFO : setting ignored attribute cum_table to None
2017-04-03 09:33:54,663 : INFO : loaded C:/Users/Yash/Documents/Project_4/imdb1.w2v
2017-04-03 09:33:55,011 : INFO : storing 53263x100 projection weights into C:/Users/Yash/Documents/Project_4/imdb_readable.w2v


In [3]:
model.wv.save_word2vec_format('resources/wordvec_pre.txt', binary=False)
import numpy as np

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.wv = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        #self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.wv[w] for w in words if w in self.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

2017-04-03 09:34:05,580 : INFO : storing 53263x100 projection weights into C:/Users/Yash/Documents/Project_4/wordvec_pre.txt


In [4]:
w2v = MeanEmbeddingVectorizer(model)
y_train = []
y_test = []
X_train = sentences[ : 25000]
X_test = sentences[25000:50000]

for i in range(12500):
    y_train.append('positive')
    y_test.append('positive')

for i in range(12500):
    y_train.append('negative')
    y_test.append('negative')
    
X_train = w2v.transform(X_train)
X_test=w2v.transform(X_test)
print(X_train.shape)
print(X_train)

(25000, 100)
[[ 0.78009146  0.87203753 -0.53153795 ...,  0.38531545  0.89650971
  -0.34766012]
 [ 0.87291992  0.81601912 -0.7513985  ...,  0.25126147  0.70795953
  -0.03906186]
 [ 0.44415048  0.71300572 -0.51766372 ...,  0.22169966  0.45642874
  -0.04845271]
 ..., 
 [ 0.69752133  1.08358967 -0.46106651 ...,  0.3044163   0.64457202
  -0.60528517]
 [ 0.37689963  0.9939642  -0.42636824 ...,  0.41567418  0.72116739
  -0.37780032]
 [ 0.41763598  1.3784945  -0.85002297 ...,  0.37775347  0.63217473
  -0.90100771]]


In [5]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


model_LR1 = LogisticRegression()
print('Logistic Regression Classifier Training starts ...')
model_LR1 = model_LR1.fit(X_train, y_train)
print('Logistic Regression Classifier Training is done and transorming testing data using trained tf-idf ...')



y_pred1 = model_LR1.predict(X_test)

print('Testing Data Prediction Done!!!')

print(y_pred1)

print(accuracy_score(y_test, y_pred1))

Logistic Regression Classifier Training starts ...
Logistic Regression Classifier Training is done and transorming testing data using trained tf-idf ...
Testing Data Prediction Done!!!
['positive' 'positive' 'positive' ..., 'positive' 'positive' 'negative']
0.85384


In [6]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

model_lSVC = LinearSVC()
print('Support Vector Machine Classifier Training starts ...')
model_lSVC.fit(X_train, y_train)  
print('Support Vector Machine Classifier Training is done and transorming testing data using trained tf-idf ...')


y_pred = model_lSVC.predict(X_test)
print('Testing Data Prediction Done!!!')
print(y_pred.shape)
print(y_pred)
print(accuracy_score(y_test, y_pred))

Support Vector Machine Classifier Training starts ...
Support Vector Machine Classifier Training is done and transorming testing data using trained tf-idf ...
Testing Data Prediction Done!!!
(25000,)
['positive' 'positive' 'positive' ..., 'positive' 'positive' 'negative']
0.8536


In [7]:
from sklearn.linear_model import SGDClassifier

model_SGD = SGDClassifier()
print('Stochastic Gradient Descent Classifier Training starts ...')
model_SGD.fit(X_train, y_train)
print('Stochastic Gradient Descent Classifier Training is done and transorming testing data using trained tf-idf ...')


y_pred = model_SGD.predict(X_test)
print('Testing Data Prediction Done!!!')
print(y_pred.shape)
print(y_pred)
print(accuracy_score(y_test, y_pred))

Stochastic Gradient Descent Classifier Training starts ...
Stochastic Gradient Descent Classifier Training is done and transorming testing data using trained tf-idf ...
Testing Data Prediction Done!!!
(25000,)
['positive' 'positive' 'positive' ..., 'negative' 'positive' 'negative']
0.84608


In [8]:
from sklearn import tree

model_DT = tree.DecisionTreeClassifier(criterion='gini')
print('Decision Tree Classifier using gini index Training starts ...')
model_DT = model_DT.fit(X_train, y_train)
print('Decision Tree Classifier Training is done and transorming testing data using trained tf-idf ...')


y_pred = model_DT.predict(X_test)
print('Testing Data Prediction Done!!!')
print(y_pred.shape)
print(y_pred)
print(accuracy_score(y_test, y_pred))

Decision Tree Classifier using gini index Training starts ...
Decision Tree Classifier Training is done and transorming testing data using trained tf-idf ...
Testing Data Prediction Done!!!
(25000,)
['positive' 'positive' 'positive' ..., 'positive' 'positive' 'negative']
0.68472


In [11]:
from sklearn.ensemble import GradientBoostingClassifier

model_GBC = GradientBoostingClassifier(n_estimators=1000, learning_rate=1.0, max_depth=1, random_state=0)
print('Gradient Boosting Classifier using 10 n_estimatores Training starts ...')
model_GBC = model_GBC.fit(X_train, y_train)
print('Gradient Boosting Classifier Training is done and transorming testing data using trained tf-idf ...')


y_pred = model_GBC.predict(X_test)
print('Testing Data Prediction Done!!!')
print(y_pred.shape)
print(y_pred)

print(accuracy_score(y_test, y_pred))

Gradient Boosting Classifier using 10 n_estimatores Training starts ...
Gradient Boosting Classifier Training is done and transorming testing data using trained tf-idf ...
Testing Data Prediction Done!!!
(25000,)
['positive' 'positive' 'positive' ..., 'positive' 'positive' 'positive']
0.8352
